In [21]:
import boto3
import pandas as pd

In [3]:
# check if boto3 is installed
boto3.__version__

'1.26.15'

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()
#environment variables
ACCESS_KEY_ID = os.getenv("ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")



In [5]:
s3= boto3.client('s3',
                 region_name='eu-central-1',
                 aws_access_key_id=f'{ACCESS_KEY_ID}',
                 aws_secret_access_key=f'{AWS_SECRET_KEY}')

In [6]:
respose = s3.list_buckets()
for bucket in respose['Buckets']:
    print(bucket)

{'Name': 'my-first-bucket-128341164379', 'CreationDate': datetime.datetime(2022, 11, 24, 14, 2, 29, tzinfo=tzutc())}
{'Name': 'samil-aws-config', 'CreationDate': datetime.datetime(2022, 11, 24, 11, 11, 58, tzinfo=tzutc())}


In [77]:
bucket = s3.create_bucket(Bucket='my-first-bucket-128341164379',
                          CreateBucketConfiguration={'LocationConstraint': 'eu-central-1'})

- Get the buckets from S3.
- Iterate over the bucket key from response to access the list of buckets.
- Print the name of each bucket.

In [8]:
response = s3.list_buckets()
for bucket in response['Buckets']:
    print(bucket['Name'], bucket['CreationDate'])

my-first-bucket-128341164379 2022-11-24 14:02:29+00:00
samil-aws-config 2022-11-24 11:11:58+00:00


In [70]:
response = s3.delete_bucket(Bucket='my-first-bucket-128341164379')

In [9]:
# Delete all the buckets with 'gim', create replacements.
for bucket in response['Buckets']:
  if 'gim' in bucket['Name']:
      s3.delete_bucket(Bucket=bucket['Name'])

In [15]:
# Upload a new file.
# Filename is the local file path.
# Bucket is the name of the bucket.
# Key is what we want to call the object in S3.
s3.upload_file(Filename= '/Users/ssamilozkan/Desktop/pyspark/airports.csv' ,
               Bucket= 'my-first-bucket-128341164379' , 
               Key= 'aiport.csv' )

In [16]:
# List all the objects in the bucket.
response = s3.list_objects(Bucket= 'my-first-bucket-128341164379', MaxKeys=2)
for obj in response['Contents']:
    print(obj['Key'], obj['Size'], obj['LastModified'])

aiport.csv 84548 2022-11-25 13:53:36+00:00
planes.csv 219326 2022-11-25 13:53:15+00:00


In [85]:
# Download the file.
# You can see the new file in the current directory.
s3.download_file(Filename= '/Users/ssamilozkan/Desktop/pyspark/airports2.csv' ,
                 Bucket = 'my-first-bucket-128341164379' ,
                 Key = 'aiport.csv' )

In [ ]:
# Deleteing objects
s3.delete_object(Bucket= 'my-first-bucket-128341164379' , Key= 'aiport.csv' )

In [17]:
# object metadata
s3.head_object(Bucket= 'my-first-bucket-128341164379' , Key= 'aiport.csv' )

{'ResponseMetadata': {'RequestId': 'XKMSR3GCZ2R03XZA',
  'HostId': 'owokeTx7VACJAehcJjyjvBJ4rjkK3pmu6OMHvKc3q9tM9lTe4Kfyrdv7/U7tTlQso1AiQciT+3I=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'owokeTx7VACJAehcJjyjvBJ4rjkK3pmu6OMHvKc3q9tM9lTe4Kfyrdv7/U7tTlQso1AiQciT+3I=',
   'x-amz-request-id': 'XKMSR3GCZ2R03XZA',
   'date': 'Fri, 25 Nov 2022 13:53:42 GMT',
   'last-modified': 'Fri, 25 Nov 2022 13:53:36 GMT',
   'etag': '"a31ea87f5412c40ed4907c38478e5c80"',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'server': 'AmazonS3',
   'content-length': '84548'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2022, 11, 25, 13, 53, 36, tzinfo=tzutc()),
 'ContentLength': 84548,
 'ETag': '"a31ea87f5412c40ed4907c38478e5c80"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {}}

In [ ]:
# List only objects that start with '2018/final_'
response = s3.list_objects(Bucket='gid-staging', 
                           Prefix='2018/final_')

# Iterate over the objects
if 'Contents' in response:
  for obj in response['Contents']:
      # Delete the object
      s3.delete_object(Bucket='gid-staging', Key=obj['Key'])

# Print the remaining objects in the bucket
response = s3.list_objects(Bucket='gid-staging')

for obj in response['Contents']:
  	print(obj['Key'])

In [18]:
#ACL (Access Control List)
# set the ACL to public-read
# Now anyone can read the object
s3.put_object_acl(Bucket='my-first-bucket-128341164379', Key = 'aiport.csv', ACL='public-read')

{'ResponseMetadata': {'RequestId': 'YH0PJBBHN0Y9CGNR',
  'HostId': 'CAROSxLvznLbaflRjsLYY8uPDHHi7wnhDOQ6GU2lhSgRNVgc2OeKRrA3t1AjyFdnB63LiMfxnDg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'CAROSxLvznLbaflRjsLYY8uPDHHi7wnhDOQ6GU2lhSgRNVgc2OeKRrA3t1AjyFdnB63LiMfxnDg=',
   'x-amz-request-id': 'YH0PJBBHN0Y9CGNR',
   'date': 'Fri, 25 Nov 2022 13:53:51 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [25]:
# Setting the ACLs on upload
# upload file with public-read ACL
s3.upload_file(Filename= '/Users/ssamilozkan/Desktop/pyspark/pyspark_tutorial/dataset/planes.csv' ,
               Bucket= 'my-first-bucket-128341164379' ,
               Key= 'planes.csv' ,
               ExtraArgs={'ACL': 'public-read'})

-  **Accessing the public objects**
- URL for Key = `https://{bucket}.{key}` 
- Example: `https://my-first-bucket-128341164379.planes.csv`


In [28]:
# Generating public object URL
url = "https://{}.s3.{}.amazonaws.com/{}".format('my-first-bucket-128341164379', 'eu-central-1','planes.csv')

In [30]:
# read the URL into a pandas dataframe
df = pd.read_csv(url)
df.head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
1,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N105UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N107US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan


In [32]:
# Set the ACL to private
s3.put_object_acl(Bucket = 'my-first-bucket-128341164379', Key = 'planes.csv', ACL='private')

{'ResponseMetadata': {'RequestId': '7N1NDNESS9JR9SGJ',
  'HostId': 'PKy7DxRV7plrndh9RTIja9YsjTj1ExzXn694AuRNwXbi3YY8fAhyEkrYQ7J6tGw+hPponbs5qsg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'PKy7DxRV7plrndh9RTIja9YsjTj1ExzXn694AuRNwXbi3YY8fAhyEkrYQ7J6tGw+hPponbs5qsg=',
   'x-amz-request-id': '7N1NDNESS9JR9SGJ',
   'date': 'Fri, 25 Nov 2022 14:11:18 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}